### 데이터 불러오기

In [ ]:
import pandas as pd

origin_csv = pd.read_csv('data/Glucose_Set-2A-1.csv')
# df = df.sample(frac = 1)
usable_data = pd.DataFrame()
usable_data = pd.concat([origin_csv['mPD1_AL-D'], origin_csv['mPD2_AL-D'], origin_csv['mPD3_AL-D'], 
                         origin_csv['T-rPD_AL-D'], origin_csv['R-rPD_AL-D'],
                         origin_csv['LD_C_Av'], origin_csv['T-rPD'], origin_csv['Th3_C'], origin_csv['Th4_C'], origin_csv['Th6_C'],
                         origin_csv['Th3_AvD'], origin_csv['Th4_AvD'], origin_csv['Th6_AvD'],
                         origin_csv['mPD1'], origin_csv['mPD2'], origin_csv['mPD3'], origin_csv['Glucose']], axis = 1, sort = False)

usable_data = usable_data.sample(frac = 1)
usable_data.head()

### 모델 코드 복사본

In [2]:
import test_densenet
from base_densenet import DenseNet
from test_densenet import user_setting
from test_densenet import address_book

import torch
import torch.nn as nn

class CgmDenseNet(nn.Module):
    def __init__(self, classes=9, device="cuda"):
        super(CgmDenseNet, self).__init__()
        self.device = torch.device(device)
        self.var_layer = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=2, padding=1)
        self.var_layer.to(self.device)
        self.up = nn.Upsample(scale_factor=16, mode='nearest')
        
        self.densenet = DenseNet(num_classes=classes) # 0, 150, 200, 250, 300, 350, 400, 450, 500
        self.densenet.to(self.device)
    
    def forward(self, x):
        x = self.var_layer(x)
        x = x.view(1,3, 4, 4)
        
        x = self.up(x)
        # print("upsacled shape", x.shape)
        x = self.densenet(x)
        return x

In [4]:
y_data.head()

611     7
2737    6
4177    6
1640    2
4255    7
Name: Glucose, dtype: int64

### 정규화 작업, Class로 치환

In [5]:
from sklearn.preprocessing import MinMaxScaler

mMscaler = MinMaxScaler()
mMscaler.fit(usable_data)

mMscaled_data = mMscaler.transform(usable_data)
x_data = pd.DataFrame(mMscaled_data)

x_data = x_data.replace(('Glucose',1.0), 8)
x_data = x_data.replace(('Glucose',0.3), 1)
x_data = x_data.replace(('Glucose',0.4), 2)
x_data = x_data.replace(('Glucose',0.5), 3)
x_data = x_data.replace(('Glucose',0.6), 4)
x_data = x_data.replace(('Glucose',0.7000000000000001), 5)
x_data = x_data.replace(('Glucose',0.8), 6)
x_data = x_data.replace(('Glucose',0.9), 7)
# x_data = x_data.replace(('Glucose',0.7), 5)

x_data[0:10]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.443699,0.000176,0.000235,0.376498,0.386792,0.000000,0.376670,0.852980,0.720000,0.828140,0.849797,0.717833,0.827455,0.443546,0.000296,0.000369,7.0
1,0.000420,0.995528,0.000342,0.726206,0.691037,0.815534,0.725548,0.868874,0.768889,0.762040,0.855210,0.762980,0.757865,0.000289,0.994963,0.000369,6.0
2,0.000515,0.000329,0.998385,0.104184,0.167451,8.000000,0.104222,0.743046,0.675556,0.946176,0.742896,0.677201,0.957102,0.000578,0.000395,0.998647,6.0
3,0.000636,0.358962,0.000503,0.187855,0.207544,0.718447,0.187867,0.459603,0.300000,0.480642,0.456022,0.295711,0.479504,0.000578,0.358977,0.000615,2.0
4,0.000530,0.000383,0.998178,0.104136,0.220125,8.000000,0.104222,0.758940,0.726667,0.874410,0.780785,0.731377,0.878932,0.000433,0.000494,0.998032,7.0
5,0.000583,0.000360,0.530527,0.000594,0.206196,0.699029,0.000534,0.703311,0.686667,0.816808,0.717185,0.686230,0.819828,0.000578,0.000494,0.530866,8.0
6,0.000636,0.000404,0.998427,0.104392,0.231132,8.000000,0.104490,0.822517,0.733333,0.758263,0.820027,0.731377,0.763584,0.000433,0.000593,0.998032,4.0
7,0.997789,0.000117,0.000332,0.997788,0.726418,0.097087,0.997194,0.458278,0.348889,0.944287,0.466847,0.349887,0.948522,0.997401,0.000198,0.000369,6.0
8,0.000591,0.000373,0.530785,0.001150,0.103773,0.699029,0.001069,0.868874,0.826667,0.731822,0.893099,0.837472,0.735939,0.000578,0.000494,0.530866,6.0
9,0.000352,0.358594,0.000361,0.186986,0.299529,0.718447,0.187600,0.790728,0.635556,0.714825,0.791610,0.634312,0.712107,0.000289,0.359372,0.000492,3.0


### test코드 동작 

실제 출력값들의 max값 위치와 y값(class번호) 비교

In [7]:
dense_net = CgmDenseNet()
device = torch.device('cuda')
wgt_file = torch.load("property/test2.pth", map_location="cuda")
dense_net.load_state_dict(wgt_file)
dense_net.eval()

count = 0
for n in range(0, 5): # len(x_data)
    x_tensor = torch.Tensor([[x_data[0][n], x_data[1][n], x_data[2][n],
                            x_data[3][n], x_data[4][n], x_data[5][n],
                            x_data[6][n], x_data[7][n], x_data[8][n],
                            x_data[9][n], x_data[10][n], x_data[11][n],
                            x_data[12][n], x_data[14][n], x_data[15][n]
                            ]]).to("cuda")
    y_tensor = torch.Tensor([x_data[16][n]]).to("cuda")
    result = dense_net(x_tensor)
    
    print(int(y_tensor[0]), ' : ', int(torch.argmax(result, dim = 1)[0]))
    # if int(y_tensor[0]) == int(torch.argmax(result, dim = 1)):
    #     count += 1
    
    if n != 0 and (n % 500) == 0:
        print(count/n)

# print(count/len(x_data))

7  :  1
6  :  1
6  :  8
2  :  1
7  :  8
